In [ ]:
import data_analysis.data_processing as dp
import data_analysis.data_learning as dl

# reload module to bypass caching
import importlib
importlib.reload(dp)
importlib.reload(dl)

# paths
test_window_path_home = dp.Path(r'\\?\C:\Users\jannis\Documents\HSP_IDS\Material\Aktuell\2025-02-17_11-14-33_192.168.1.0-normal_1\1554220324.748197-1554220354.748197') # treat it as a long path to avoid path length issues on windows
test_window_path_remote = dp.Path(r'/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff/2025-03-04_00-03-20_192.168.1.0-normal_DDoS_1/1556203726.876922-1556203756.876922')

attack_data_set_path = dp.Path(r'/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff')
ddos_test_path_parquet = dp.Path(r'/home/hsp252/Development/DDoS')

feature_vectors = dp.pd.DataFrame()
tsw_paths = dp.getThirtySecondWindowPaths(attack_data_set_path)
NR_OF_FVS = 100

# create ddos feature vectors
# selected_cols = dp.pd.read_csv('/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff/2025-02-28_08-41-32_192.168.1.0-normal_backdoor/1556466432.434372-1556466462.434372/192.168.1.152/connections/1556466432.434372-1556466442.434372/96.0_192.168.1.193_49338.0_192.168.1.152_1880.0/host_data_chunk_full.csv').select_dtypes(include='number').columns.to_list() \
# + dp.pd.read_csv('/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff/2025-02-28_08-41-32_192.168.1.0-normal_backdoor/1556466432.434372-1556466462.434372/192.168.1.152/s2_selected_qs.csv').select_dtypes(include='number').columns.to_list() \
# + dp.pd.read_csv('/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff/2025-02-28_08-41-32_192.168.1.0-normal_backdoor/1556466432.434372-1556466462.434372/192.168.1.152/s3_connection_qs.csv').select_dtypes(include='number').columns.to_list() \
# + dp.pd.read_csv('/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff/2025-02-28_08-41-32_192.168.1.0-normal_backdoor/1556466432.434372-1556466462.434372/s1_general_qs.csv').select_dtypes(include='number').columns.to_list()

# test_ddos_feature_vectors = dl.create_test_ddos_feature_vectors(ddos_test_path_parquet, selected_cols)

# test_ddos_feature_vectors_path = dp.Path(r'/home/hsp252/Development/HSP_IDS/test_ddos_df.pkl')
# dl.save_to_pickle(test_ddos_feature_vectors, test_ddos_feature_vectors_path)

ddos_feature_vectors = dp.pp.load('test_ddos_df.pkl')
valid_columns = ddos_feature_vectors.columns.to_list()
ddos_feature_vectors['attack_type'] = 'ddos'
feature_vectors = dp.pd.concat([feature_vectors, ddos_feature_vectors])
print(ddos_feature_vectors.shape)

# create mitm feature vectors and store
runsomware_feature_vectors = dp.pd.DataFrame()
for index, tsw_object in tsw_paths[tsw_paths['type'] == 'mitm'].iterrows():
    start = dp.time.time()
    tsw = dp.ThirtySecondWindow(dp.Path(tsw_object['path']))
    mitm_local_feature_vectors = dl.create_feature_vectors(tsw, valid_columns)
    mitm_local_feature_vectors['attack_type'] = tsw_object['type'] # add attack type column for training
    runsomware_feature_vectors = dp.pd.concat([runsomware_feature_vectors, mitm_local_feature_vectors])
    end = dp.time.time()
    row_count = runsomware_feature_vectors.shape[0]
    print(f"created mitm feature vectors for {tsw_object['type']} in {end - start}s, rows: {row_count}")
    if row_count >= NR_OF_FVS:
        break
    #break
feature_vectors = dp.pd.concat([feature_vectors, runsomware_feature_vectors])
mitm_feature_vectors_path = dp.Path(r'/home/hsp252/Development/HSP_IDS/test_mitm_df.pkl')
dl.save_to_pickle(runsomware_feature_vectors, mitm_feature_vectors_path)

# create runsomware feature vectors and store
runsomware_feature_vectors = dp.pd.DataFrame()
for index, tsw_object in tsw_paths[tsw_paths['type'] == 'runsomware'].iterrows():
    start = dp.time.time()
    tsw = dp.ThirtySecondWindow(dp.Path(tsw_object['path']))
    runsomware_local_feature_vectors = dl.create_feature_vectors(tsw, valid_columns)
    runsomware_local_feature_vectors['attack_type'] = tsw_object['type'] # add attack type column for training
    runsomware_feature_vectors = dp.pd.concat([runsomware_feature_vectors, runsomware_local_feature_vectors])
    end = dp.time.time()
    row_count = runsomware_feature_vectors.shape[0]
    print(f"created runsomware feature vectors for {tsw_object['type']} in {end - start}s, rows: {row_count}")
    if row_count >= NR_OF_FVS:
        break
    #break
feature_vectors = dp.pd.concat([feature_vectors, runsomware_feature_vectors])
runsomware_feature_vectors_path = dp.Path(r'/home/hsp252/Development/HSP_IDS/test_runsomware_df.pkl')
dl.save_to_pickle(runsomware_feature_vectors, runsomware_feature_vectors_path)

# create injection feature vectors and store
injection_feature_vectors = dp.pd.DataFrame()
for index, tsw_object in tsw_paths[tsw_paths['type'] == 'injection'].iterrows():
    start = dp.time.time()
    tsw = dp.ThirtySecondWindow(dp.Path(tsw_object['path']))
    injection_local_feature_vectors = dl.create_feature_vectors(tsw, valid_columns)
    injection_local_feature_vectors['attack_type'] = tsw_object['type'] # add attack type column for training
    injection_feature_vectors = dp.pd.concat([injection_feature_vectors, injection_local_feature_vectors])
    end = dp.time.time()
    row_count = injection_feature_vectors.shape[0]
    print(f"created injection feature vectors for {tsw_object['type']} in {end - start}s, rows: {row_count}")
    if row_count >= NR_OF_FVS:
        break
    #break
feature_vectors = dp.pd.concat([feature_vectors, injection_feature_vectors])
injection_feature_vectors_path = dp.Path(r'/home/hsp252/Development/HSP_IDS/test_injection_df.pkl')
dl.save_to_pickle(injection_feature_vectors, injection_feature_vectors_path)

# create backdoor feature vectors and store
backdoor_feature_vectors = dp.pd.DataFrame()
for index, tsw_object in tsw_paths[tsw_paths['type'] == 'backdoor'].iterrows():
    start = dp.time.time()
    tsw = dp.ThirtySecondWindow(dp.Path(tsw_object['path']))
    backdoor_local_feature_vectors = dl.create_feature_vectors(tsw, valid_columns)
    backdoor_local_feature_vectors['attack_type'] = tsw_object['type'] # add attack type column for training
    backdoor_feature_vectors = dp.pd.concat([backdoor_feature_vectors, backdoor_local_feature_vectors])
    end = dp.time.time()
    row_count = backdoor_feature_vectors.shape[0]
    print(f"created backdoorn feature vectors for {tsw_object['type']} in {end - start}s, rows: {row_count}")
    if row_count >= NR_OF_FVS:
        break
    #break
feature_vectors = dp.pd.concat([feature_vectors, backdoor_feature_vectors])
backdoor_feature_vectors_path = dp.Path(r'/home/hsp252/Development/HSP_IDS/test_backdoor_df.pkl')
dl.save_to_pickle(backdoor_feature_vectors, backdoor_feature_vectors_path)

# create dos feature vectors and store
dos_feature_vectors = dp.pd.DataFrame()
for index, tsw_object in tsw_paths[tsw_paths['type'] == 'dos'].iterrows():
    start = dp.time.time()
    tsw = dp.ThirtySecondWindow(dp.Path(tsw_object['path']))
    dos_local_feature_vectors = dl.create_feature_vectors(tsw, valid_columns)
    dos_local_feature_vectors['attack_type'] = tsw_object['type'] # add attack type column for training
    dos_feature_vectors = dp.pd.concat([dos_feature_vectors, dos_local_feature_vectors])
    end = dp.time.time()
    row_count = dos_feature_vectors.shape[0]
    print(f"created dos feature vectors for {tsw_object['type']} in {end - start}s, rows: {row_count}")
    if row_count >= NR_OF_FVS:
        break
    #break
feature_vectors = dp.pd.concat([feature_vectors, dos_feature_vectors])
dos_feature_vectors_path = dp.Path(r'/home/hsp252/Development/HSP_IDS/test_dos_df.pkl')
dl.save_to_pickle(dos_feature_vectors, dos_feature_vectors_path)





    
    # put that into a dataset for RFC
    # for non-DDoS stuff: acquire through 30s-windows, hosts and connections, do mean and median and add to dataset for training RFC
    # check the least amount of 30s windows in attack classes and take this amount of 30s windows for now


[06-06-2025 14:09:23] [pypickle.pypickle] [INFO] Loading Pickle file: [test_ddos_df.pkl]


(1084, 130)
created mitm feature vectors for mitm in 1.1589179039001465s, rows: 7
created mitm feature vectors for mitm in 1.0830488204956055s, rows: 11
created mitm feature vectors for mitm in 1.9867689609527588s, rows: 18
created mitm feature vectors for mitm in 1.3071486949920654s, rows: 24
created mitm feature vectors for mitm in 2.673678159713745s, rows: 30
created mitm feature vectors for mitm in 8.94458270072937s, rows: 37
created mitm feature vectors for mitm in 9.849808931350708s, rows: 42
created mitm feature vectors for mitm in 3.3192477226257324s, rows: 46
created mitm feature vectors for mitm in 7.654247283935547s, rows: 51
created mitm feature vectors for mitm in 4.584762334823608s, rows: 55
created mitm feature vectors for mitm in 2.0147202014923096s, rows: 59
created mitm feature vectors for mitm in 2.4382259845733643s, rows: 65
created mitm feature vectors for mitm in 6.654208421707153s, rows: 72
created mitm feature vectors for mitm in 4.647281169891357s, rows: 77
cre

KeyboardInterrupt: 